<a href="https://colab.research.google.com/github/CassiaFa/Segmentation_lesion_pulmonaire/blob/main/Segmentation_lesion_pulmonaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1 : Base de données

## Importation des packages

## Chargement des images

## Redimensionnage des images

## Diviser les données en train/validation/test

## Visualisation des données et vérité terrain associé

## Data Augmentation

# Partie 2 : Transfert Learning